CrossInTray Function 
=====================


In [11]:
using Pkg
Pkg.activate("../../.")
using Globtim
# include("../../../src/lib_func.jl") # Include the library of functions

# Constants and Parameters
d = 2 # Initial Degree 
const n, a, b = 2, 10, 1 
const scale_factor = a / b       # Scaling factor appears in `main_computation`, maybe it should be a parameter.
const delta, alpha = .5 , 1 / 10  # Sampling parameters
const tol_l2 = 1e-2            # Define the tolerance for the L2-norm

f = CrossInTray # Objective function

  Activating project at `~/Globtim.jl`


CrossInTray (generic function with 1 method)

In [12]:
while true # Potential infinite loop
    global poly_approx = MainGenerate(f, 2, d, delta, alpha, scale_factor, 0.2) # computes the approximant in Chebyshev basis
    if poly_approx.nrm < tol_l2
        println("attained the desired L2-norm: ", poly_approx.nrm)
        break
    else
        println("current L2-norm: ", poly_approx.nrm)
        println("Number of samples: ", poly_approx.N)
        println("Current degree: ", d)
        global d += 1
    end
end;

current L2-norm: 0.6430142328608683
Number of samples: 8
Current degree: 2
current L2-norm: 0.46901359098832557
Number of samples: 12
Current degree: 3
current L2-norm: 0.042265307871778346
Number of samples: 17
Current degree: 4
current L2-norm: 0.21877316208032135
Number of samples: 22
Current degree: 5
current L2-norm: 0.1377083991915426
Number of samples: 28
Current degree: 6
current L2-norm: 0.10505309468453336
Number of samples: 34
Current degree: 7
current L2-norm: 0.03621782979045448
Number of samples: 41
Current degree: 8
current L2-norm: 0.07094406778061033
Number of samples: 48
Current degree: 9
current L2-norm: 0.05115099061623992
Number of samples: 56
Current degree: 10
current L2-norm: 0.019432636381024184
Number of samples: 65
Current degree: 11
current L2-norm: 0.03512857325759277
Number of samples: 74
Current degree: 12
current L2-norm: 0.016815599842854596
Number of samples: 83
Current degree: 13
current L2-norm: 0.014656516853746946
Number of samples: 93
Current degr

In [14]:
using DynamicPolynomials, DataFrames
using HomotopyContinuation, ProgressLogging

@polyvar(x[1:n]) # Define polynomial ring 
pol = main_nd(x, n, d, poly_approx.coeffs) 

grad = differentiate.(pol, x)
sys = System(grad)
println("The system is of degree: ", d-1)

The system is of degree: 16


In [13]:
Real_sol_lstsq = HomotopyContinuation.solve(sys)
real_pts = HomotopyContinuation.real_solutions(Real_sol_lstsq; only_real=true, multiple_results=false);

Tracking 100 paths...   2%|▋                            |  ETA: 0:07:49



Tracking 100 paths...  66%|███████████████████▏         |  ETA: 0:00:05



Tracking 100 paths... 100%|█████████████████████████████| Time: 0:00:09
  # paths tracked:                  100
  # non-singular solutions (real):  100 (84)
  # singular endpoints (real):      0 (0)
  # total solutions (real):         100 (84)


In [6]:
condition(point) = -1 < point[1] < 1 && -1 < point[2] < 1
filtered_points = filter(condition, real_pts) # Filter points using the filter function
# Colllect the critical points of the approximant 
h_x = Float64[point[1] for point in filtered_points] # Initialize the x vector for critical points of approximant
h_y = Float64[point[2] for point in filtered_points] # Initialize the y vector
h_z = map(p -> f([p[1], p[2]]), zip(scale_factor * h_x, scale_factor * h_y))
df = DataFrame(x=scale_factor * h_x, y=scale_factor * h_y, z=h_z); # Create a DataFrame

In [7]:
using GLMakie
# Extract coordinates and function values
coords = poly_approx.scale_factor * poly_approx.grid
z_coords = poly_approx.z;

In [8]:
if size(coords)[2] == 2  # Plot the 3D scatter plot if the dimensions are 2
    fig = Figure(size=(800, 600))
    ax = Axis3(fig[1, 1], title="Cross In Tray Sample Points", xlabel="X-axis", ylabel="Y-axis", zlabel="Z-axis")

    # Plot the sampled data with a darker core
    scatter!(ax, coords[:, 1], coords[:, 2], z_coords, markersize=2, color=:black, label="Sampled Data Core")

    # Plot the sampled data with adjusted size and color to create a halo effect
    scatter!(ax, coords[:, 1], coords[:, 2], z_coords, markersize=4, color=z_coords, colormap=:viridis, label="Sampled Data Halo")

    # Plot the exact approximant critical points with adjusted size
    scatter!(ax, df.x, df.y, df.z, markersize=10, color=:red, label="Exact approximant critical points")

    display(fig)
end

2024-11-29 14:48:36.938 julia[8320:78876] +[IMKClient subclass]: chose IMKClient_Modern
2024-11-29 14:48:36.938 julia[8320:78876] +[IMKInputSession subclass]: chose IMKInputSession_Modern


GLMakie.Screen(...)